In [1]:
#Creating the model for mobilnet ssd
APIMODEL_PATH = "Tensorflow/models"
SCRIPTS_PATH = "Tensorflo/scripts"
WORKSPACE_PATH = "Tensorflow/workspace"
ANNOTATIONS_PATH = WORKSPACE_PATH + "/annotations"
IMAGES_PATH = "Tensorflow/workspace/images"
MODEL_PATH = "Tensorflow/workspace/models"
PRE_TRAINED_MODEL_PATH = "Tensorflow/workspace/pre-trained-models"
CHECKPOINT_PATH = MODEL_PATH + "/Mobilenet640"


In [4]:
labelmap = [{'name':'healthy','id':1},{'name':'unhealthy', 'id':2}]
import os
with open(os.path.join(ANNOTATIONS_PATH,'label_map.pbtxt'),'w') as f:

    for label in labelmap:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

print('\x1b[6;30;42m' + 'Success!' + '\x1b[0m')

Success!


In [5]:
model_name = 'Mobilenet640'

In [7]:
!mkdir Tensorflow\\workspace\\models\\Mobilenet640

A subdirectory or file Tensorflow\\workspace\\models\\Mobilenet640 already exists.


In [8]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [11]:
CONFIG_PATH = "C:\\Projects\\jopan project\\Tensorflow\\workspace\\models\\Mobilenet640\\pipeline.config"

In [12]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [17]:
#update conf file 

pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 2
pipeline_config.train_config.fine_tune_checkpoint = "C:\\Projects\\SSUPRTModelSelection\\Tensorflow\\workspace\\pre-trained-models\\ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8\\ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8\\checkpoint\\ckpt-0"
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATIONS_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATIONS_PATH + '/train.tfrecord']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATIONS_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATIONS_PATH + '/test.tfrecord']

In [18]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)  

In [19]:
pipeline_config

model {
  ssd {
    num_classes: 2
    image_resizer {
      fixed_shape_resizer {
        height: 640
        width: 640
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
 

In [20]:
#Training sequence

print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,model_name,MODEL_PATH,model_name))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/Mobilenet640 --pipeline_config_path=Tensorflow/workspace/models/Mobilenet640/pipeline.config --num_train_steps=5000


In [4]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [11]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-4')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    print(detections)
    return detections

In [12]:
import cv2 
import numpy as np

In [17]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATIONS_PATH+'/label_map.pbtxt',use_display_name=True)

In [14]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))